# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

# Project: Federated Learning with a Trusted Aggregator

In [1]:
# try this project here!
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim

W0715 09:01:38.149187 4321358720 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'


W0715 09:01:38.188709 4321358720 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [2]:
bob           = sy.VirtualWorker(hook, id='bob')
alice         = sy.VirtualWorker(hook, id='alice')
secure_worker = sy.VirtualWorker(hook, id='secure_worker')

In [3]:
bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

W0715 09:01:42.981767 4321358720 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior


W0715 09:01:42.985526 4321358720 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior


W0715 09:01:42.987744 4321358720 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


W0715 09:01:42.989317 4321358720 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior


W0715 09:01:42.991110 4321358720 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior


W0715 09:01:42.995395 4321358720 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure_worker #objects:0>

In [4]:
# Dataset
data   = th.tensor([[0.,0], [0,1], [1,0], [1,1]], requires_grad=True)
target = th.tensor([[0.], [0], [1], [1]], requires_grad=True)

In [5]:
bobs_data   = data[:2].send(bob)
bobs_target = target[:2].send(bob)

In [6]:
alices_data   = data[2:].send(alice)
alices_target = target[2:].send(alice)

In [7]:
# model
model = nn.Linear(2,1)

In [8]:
def model_learning(obj_model, obj_data, obj_target):
    obj_opt  = optim.SGD(params=bobs_model.parameters(),   lr=0.1)
    
    obj_opt.zero_grad()
    obj_pred = obj_model(obj_data)
    
    obj_loss = ((obj_pred - obj_target)**2).sum()
    obj_loss.backward()
    
    obj_opt.step()
    
    return obj_loss.get().data

In [9]:
 for round_iter in range(10):
    bobs_model   = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bob_loss   = th.FloatTensor
    alice_loss = th.FloatTensor

    for i in range(10):
        bob_loss   = model_learning(bobs_model,   bobs_data,   bobs_target)
        alice_loss = model_learning(alices_model, alices_data, alices_target)
    else:
        print('Bob loss:', str(bob_loss), 'Alice loss:', str(alice_loss))

    bobs_model.move(secure_worker)
    alices_model.move(secure_worker)

    with th.no_grad():
        model.weight.set_(((alices_model.weight.data + bobs_model.weight.data)/2).get())
        model.bias.set_(((alices_model.bias.data + bobs_model.bias.data)/2).get())

Bob loss: tensor(0.0127) Alice loss: tensor(0.5655)
Bob loss: tensor(0.0067) Alice loss: tensor(1.2973)


Bob loss: tensor(0.0035) Alice loss: tensor(1.8789)
Bob loss: tensor(0.0019) Alice loss: tensor(2.2186)


Bob loss: tensor(0.0010) Alice loss: tensor(2.3971)
Bob loss: tensor(0.0005) Alice loss: tensor(2.4858)


Bob loss: tensor(0.0003) Alice loss: tensor(2.5281)


Bob loss: tensor(0.0001) Alice loss: tensor(2.5474)


Bob loss: tensor(7.5413e-05) Alice loss: tensor(2.5556)


Bob loss: tensor(3.9733e-05) Alice loss: tensor(2.5585)


# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive. 

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [10]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [11]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [12]:
bob_x_share = 2 * 2
alice_x_share = 3 * 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

This even works for addition between two shared values!!

In [13]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

As you can see, we just added two numbers together while they were still encrypted!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant. I.e.

In [14]:
x = 5

Q = 23740629843760239486723

bob_x_share = 23552870267 # <- a random number
alice_x_share = Q - bob_x_share + x
alice_x_share

23740629843736686616461

In [15]:
(bob_x_share + alice_x_share) % Q

5

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so.

In [16]:
x_share = (2,5,7)

Even though normally those shares would be distributed amongst several workers, you can store them in ordered tuples like this for now :)

In [17]:
# try this project here!
import random


Q = 23740629843760239486723

def encrypt(x, n_share=3):
    shares = []
    
    for i in range(n_share):
        shares.append(random.randrange(Q))
    
    final_share = Q - ( sum(shares)%Q ) + x
    
    shares.append(final_share)
    
    return tuple(shares)
    
encrypt(5, n_share=10)

(3051834311018479997529,
 9819760213336193381140,
 10520840102577745440454,
 17766900969492401857980,
 12688554019842218315889,
 18787566258464343729633,
 14351927934289175982667,
 8479320884927729371310,
 3535279199422162301890,
 12697380314785103472052,
 7003785010645643583076)

In [18]:
def decrypt(shares):
    return sum(shares) % Q

In [19]:
def add(a, b):
    c = []
    
    assert( len(a) == len(b) )
    
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
    
    return tuple(c)

In [20]:
decrypt(add(encrypt(5), encrypt(10)))

15

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [21]:
BASE=10
PRECISION=4
Q = 23740629843760239486723

In [22]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [23]:
encode(2.5)

25000

In [24]:
decode(25000)

2.5

In [25]:
x = encrypt(encode(5.51))
y = encrypt(encode(2.38))
z = add(x,y)
decode(decrypt(z))

7.89

# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [26]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [27]:
x = th.tensor([1,2,3,4,5])

### Secret Sharing Using PySyft

We can share using the simple .share() method!

In [28]:
x = x.share(bob, alice, secure_worker)

In [29]:
bob._objects

{37199771479: tensor([1542513144430890851, 1389193617542268405, 3573857022867483725,
          631640686731858028, 2674632924853659411])}

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [30]:
y = x + x

In [31]:
y

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:46057568097 -> bob:21443570171]
	-> (Wrapper)>[PointerTensor | me:50880956706 -> alice:20297988128]
	-> (Wrapper)>[PointerTensor | me:41080300759 -> secure_worker:32154762941]
	*crypto provider: me*

In [32]:
y.get()

tensor([ 2,  4,  6,  8, 10])

### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using .fix_precision()

In [45]:
x = th.tensor([0.1,0.2,0.3])

In [46]:
x

tensor([0.1000, 0.2000, 0.3000])

In [47]:
x = x.fix_prec()
x

(Wrapper)>FixedPrecisionTensor>tensor([100, 200, 300])

In [48]:
x.child.child


FixedPrecisionTensor>tensor([100, 200, 300])

In [49]:

x.child

FixedPrecisionTensor>tensor([100, 200, 300])

In [50]:
y = x + x

In [51]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000])

### Shared Fixed Precision

And of course, we can combine the two!

In [52]:
x = th.tensor([0.1, 0.2, 0.3])

In [53]:
x = x.fix_prec().share(bob, alice, secure_worker)

In [54]:
y = x + x

In [55]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])

Make sure to make the point that people can see the model averages in the clear.

# Final Project: Federated Learning with Encrypted Gradient Aggregation